<a href="https://colab.research.google.com/github/kenanmorani/Prediction_Of_Total_Produced_Biogas_Flow/blob/main/Recursive_SVM_Standard_scaled_Feature_selective_for_Biogas_Flow_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
from sklearn import preprocessing

In [143]:
# Mounting my google drive
from google.colab import drive

In [144]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [145]:
#Create DataFrame
ss = pd.read_csv('/content/gdrive/MyDrive/Biogaz_Flow/Real_Plant_Data.csv')

In [146]:
del ss['Date'] # Deleting the 'Date' column
ss.head()

,TMF,TKM_percent,TUKM_percent,Alcantine,Fatty_Accid,Total_Biogaz_Production
0,240,46.26,33.15,787.0,244.36,2113
1,120,44.77,22.02,983.0,413.54,1754
2,200,44.45,22.61,740.0,378.60,1884
3,160,44.31,26.64,761.0,353.66,1768
4,200,50.58,23.23,748.0,343.96,1715


In [147]:
d = preprocessing.minmax_scale(ss) # Applying normalization on the data

In [148]:
d

array([[0.07906977, 0.35562731, 0.49103836, 0.25709628, 0.13314154,
        0.07240799],
       [0.02325581, 0.34417282, 0.3261739 , 0.32112534, 0.22532065,
        0.05513592],
       [0.06046512, 0.34171279, 0.33491335, 0.24174237, 0.2062833 ,
        0.06139043],
       ...,
       [0.34418605, 0.55004613, 0.50866538, 0.14047192, 0.03456035,
        0.25316334],
       [0.47906977, 0.76737392, 0.70967264, 0.13393834, 0.02668715,
        0.22126534],
       [0.48372093, 0.66128536, 0.61161309, 0.35552463, 0.20717142,
        0.33403897]])

In [162]:
# The scaled / normalized dataset

scaled_df = pd.DataFrame(d) 

In [150]:
scaled_df

,TMF,TKM_percent,TUKM_percent,Alcantine,Fatty_Accid,Total_Biogaz_Production
0,0.079070,0.355627,0.491038,0.257096,0.133142,0.072408
1,0.023256,0.344173,0.326174,0.321125,0.225321,0.055136
2,0.060465,0.341713,0.334913,0.241742,0.206283,0.061390
3,0.041860,0.340637,0.394608,0.248603,0.192695,0.055809
4,0.060465,0.388838,0.344097,0.244356,0.187409,0.053260
...,...,...,...,...,...,...
389,0.390698,0.360394,0.333284,0.100725,0.088485,0.265769
390,0.404651,0.293589,0.271515,0.090546,0.059526,0.238345
391,0.344186,0.550046,0.508665,0.140472,0.034560,0.253163
392,0.479070,0.767374,0.709673,0.133938,0.026687,0.221265


In [161]:
scaled_df = ss

In [152]:
df.head()

,TMF,TKM_percent,TUKM_percent,Alcantine,Fatty_Accid,Total_Biogaz_Production
0,0.079070,0.355627,0.491038,0.257096,0.133142,0.072408
1,0.023256,0.344173,0.326174,0.321125,0.225321,0.055136
2,0.060465,0.341713,0.334913,0.241742,0.206283,0.061390
3,0.041860,0.340637,0.394608,0.248603,0.192695,0.055809
4,0.060465,0.388838,0.344097,0.244356,0.187409,0.053260


In [153]:
#Function to test model performance changes with feature elimination MSE
def Mean_Square_Error(model, x_test, y_test):
	prediction = model.predict(x_test)
	print ("Mean Square error of model:", mean_squared_error(y_test, prediction))

In [154]:
#Setting a parameter for SVM model
C = 0.1

In [155]:
# Identifying the target feature by splitting the dataset
samples = df.filter(['TMF', 'TKM_percent', 'TUKM_percent', 'Alcantine', 'Fatty_Accid'])
scores = df.filter(['Total_Biogaz_Production'])

In [156]:
# Defining the number of features to investigate
nFeatures = len(df.columns) - 1

In [157]:
samples

,TMF,TKM_percent,TUKM_percent,Alcantine,Fatty_Accid
0,0.079070,0.355627,0.491038,0.257096,0.133142
1,0.023256,0.344173,0.326174,0.321125,0.225321
2,0.060465,0.341713,0.334913,0.241742,0.206283
3,0.041860,0.340637,0.394608,0.248603,0.192695
4,0.060465,0.388838,0.344097,0.244356,0.187409
...,...,...,...,...,...
389,0.390698,0.360394,0.333284,0.100725,0.088485
390,0.404651,0.293589,0.271515,0.090546,0.059526
391,0.344186,0.550046,0.508665,0.140472,0.034560
392,0.479070,0.767374,0.709673,0.133938,0.026687


In [158]:
scores

,Total_Biogaz_Production
0,0.072408
1,0.055136
2,0.061390
3,0.055809
4,0.053260
...,...
389,0.265769
390,0.238345
391,0.253163
392,0.221265


In [159]:
rfeIndex = nFeatures

In [160]:
#Recursively eliminate features based on the lowest weight
while True:
	#Split into training and testing
	x_train, x_test, y_train, y_test = train_test_split(samples, scores, test_size = 0.50, train_size=0.50)
	
	#Create SVM model using a linear kernel
	model = svm.SVR(kernel='linear', C=C).fit(x_train, y_train)
	coef = model.coef_

	#Print co-efficients of features
	for i in range(0, nFeatures):
		print(samples.columns[i-1],":", coef[0][i-1])
	
	#Find the minimum weight among features and eliminate the feature with the smallest weight
	min = coef[0][0]
	index = 0
	for i in range(0, rfeIndex):
		if min > coef[0][i-1]:
			index = index + 1
			min = coef[0][i-1]
	if len(samples.columns) == 1:
		print("After recursive elimination we have the", samples.columns[index], "feature with a score of:", min)
		Mean_Square_Error(model, x_test, y_test)
		break
	else:
		print ("Lowest feature weight is for", samples.columns[index], "with a value of:", min)
		print ("Dropping feature", samples.columns[index])  

		#Drop the feature in the 'samples' dataframe based on the lowest feature index
		samples.drop(samples.columns[index], axis = 1, inplace = True)
		Mean_Square_Error(model, x_test, y_test)
		print ("\n")
		rfeIndex = rfeIndex - 1
		nFeatures = nFeatures - 1

Fatty_Accid : 0.0962146398788743
TMF : 0.46831207076526243
TKM_percent : 0.06884210434979215
TUKM_percent : 0.02760629007962156
Alcantine : -0.04749003594245689
Lowest feature weight is for Fatty_Accid with a value of: -0.04749003594245689
Dropping feature Fatty_Accid
Mean Square error of model: 0.020101501755622607


Alcantine : -0.016090578729035715
TMF : 0.4443156956692785
TKM_percent : 0.05531994244651067
TUKM_percent : 0.060833963396424294
Lowest feature weight is for TKM_percent with a value of: -0.016090578729035715
Dropping feature TKM_percent
Mean Square error of model: 0.02235756607662054


Alcantine : 0.012669737901112777
TMF : 0.47216410779118306
TUKM_percent : 0.16542570833251546
Lowest feature weight is for TUKM_percent with a value of: 0.012669737901112777
Dropping feature TUKM_percent
Mean Square error of model: 0.017313301918921074


Alcantine : -0.02398403961351497
TMF : 0.4339599135688139
Lowest feature weight is for Alcantine with a value of: -0.02398403961351497
Dr

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin